In [122]:
import numpy as np

My project is on finding the ideal location to place a pastry shop in New York City. I made an excel spreadhseet using information that I found online about the neighborhoods in each borough with the cheapest $USD per square foot of area. This is important as cost is always a factor in regards to where to build the restaurant. Here is the link from which I made the table from:

Retrieved 21 August 2019 from:
https://www.zumper.com/blog/2019/05/nyc-by-square-foot-see-which-neighborhood-gets-you-the-most-space-for-your-money/

I have listed the 5 neighborhoods with the cheapest $ per Square Foot per borough (Manhattan, Queens, Staten Island, The Bronx, Brooklyn). Below I converted the excel to a csv and then to a dataframe.

In [123]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share your notebook.
client_c780d1e89b2a40a3af71f2994126ca2e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='-z3fg8zswepwCyzm8ilvDEZNamANmWLOrJSI9FXZsNPw',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_c780d1e89b2a40a3af71f2994126ca2e.get_object(Bucket='mydatascienceproject-donotdelete-pr-l1ev405yhlftzm',Key='CHEAPEST AREAS IN EACH BOROUGH - Sheet1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1


,BOROUGH,NEIGHBORHOOD,$ PER SQ FT
0,MANHATTAN,Washington Heights,3.18
1,MANHATTAN,Central Harlem,3.41
2,MANHATTAN,East Harlem,3.76
3,MANHATTAN,West Harlem,4.10
4,MANHATTAN,Little Italy,4.90
5,BROOKLYN,Dyker Heights,1.82
6,BROOKLYN,Marine Park,1.87
7,BROOKLYN,Brownsville,1.98
8,BROOKLYN,Gravesend,2.10
9,BROOKLYN,Flatlands,2.10


In [124]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: | ^C
failed

CondaError: KeyboardInterrupt

Solving environment: / ^C
failed

CondaError: KeyboardInterrupt

Folium installed
Libraries imported.


In [60]:
CLIENT_ID = '2AK23ZF2WPOOHGDPFXIMIV0YKEJI0N3CKE0ENHIQBYV2QKD3' # your Foursquare ID
CLIENT_SECRET = 'X5V42VABELHKXMVRZ05L2CVXQG53EBKQVEB1SMAZF1X2H3TE' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 300
search_query = 'pastry'
radius = 200000
print(search_query + ' .... OK!')

pastry .... OK!


I've made the search query pastry. I am going to make a call to the foursquare api for every borough in NYC for pastries. Then I will make a dataframe of pastry shops in each borough. Once I have done that, I will merge all dataframes and plot the pastry shops throughout NYC on a map.

In [61]:
address = 'Brooklyn'

geolocator1 = Nominatim(user_agent="foursquare_agent")
location1 = geolocator1.geocode(address)
latitude1 = location1.latitude
longitude1 = location1.longitude
print(latitude1, longitude1)


40.6501038 -73.9495823


In [62]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/search?client_id=2AK23ZF2WPOOHGDPFXIMIV0YKEJI0N3CKE0ENHIQBYV2QKD3&client_secret=X5V42VABELHKXMVRZ05L2CVXQG53EBKQVEB1SMAZF1X2H3TE&ll=40.6501038,-73.9495823&v=20180604&query=pastry&radius=200000&limit=300'

In [65]:
results1 = requests.get(url).json()
venues1 = results1['response']['venues']
# tranform venues into a dataframe
dataframe1 = json_normalize(venues)


In [66]:
filtered_columns1 = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered1 = dataframe1.loc[:, filtered_columns1]

# function. It extracts the categories of the different venues, in this case, pastry shops in Brooklyn.
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered1['categories'] = dataframe_filtered1.apply(get_category_type, axis=1)

dataframe_filtered1.columns = [column.split('.')[-1] for column in dataframe_filtered1.columns]

#filtered columns to only include results from Brooklyn and also cleaned up dataset
indexNames = dataframe_filtered1[dataframe_filtered1['city'] != "Brooklyn" ].index
dataframe_filtered1.drop(indexNames, inplace=True)
dataframe_filtered1=dataframe_filtered1[['name', 'categories', 'address', 'city', 'country', 'distance', 'lat', 'lng']]

dataframe_filtered1

,name,categories,address,city,country,distance,lat,lng
0,Court Pastry Shop,Bakery,298 Court St,Brooklyn,United States,5407,40.684067,-73.995377
1,Aliotta Pastry Shop,Bakery,4522 Avenue N,Brooklyn,United States,3836,40.618712,-73.930852
4,Tasty Pastry,Bakery,8218 13th Ave,Brooklyn,United States,6498,40.616052,-74.012058
6,Circo's Pastry Shop,Bakery,312 Knickerbocker Ave,Brooklyn,United States,6141,40.701525,-73.923215
8,"Jean Danet Pastry, Cafe and Brick Oven Pizza",Bakery,7526 5th Ave,Brooklyn,United States,6638,40.628839,-74.023006
9,Peter Pan Donut & Pastry Shop,Donut Shop,727 Manhattan Ave,Brooklyn,United States,8463,40.726102,-73.952252
11,I.M. Pastry Studio,Bakery,1131 Nostrand Ave,Brooklyn,United States,1098,40.659950,-73.950523
13,Circo's Pastry Shop,None,312 Knickerbocker Ave,Brooklyn,United States,1385,40.640000,-73.940000
17,Daynes Pastry,Bakery,101 Lincoln Rd,Brooklyn,United States,1464,40.660999,-73.959297
18,Damascus Bread & Pastry Shop,Bakery,195 Atlantic Ave,Brooklyn,United States,5765,40.690047,-73.993054


In [67]:
import requests
address2 = 'Manhattan'
geolocator2 = Nominatim(user_agent="foursquare_agent")
location2 = geolocator2.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude

CLIENT_ID = '2AK23ZF2WPOOHGDPFXIMIV0YKEJI0N3CKE0ENHIQBYV2QKD3' 
CLIENT_SECRET = 'X5V42VABELHKXMVRZ05L2CVXQG53EBKQVEB1SMAZF1X2H3TE' 
LIMIT = 300
search_query2 = 'pastry'
radius2 = 250000 #length of Manhattan. I corrected for any outliers in case some results were outside Manhattan.

In [68]:
url2='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2, VERSION, search_query2, radius2, LIMIT)
url2

'https://api.foursquare.com/v2/venues/search?client_id=2AK23ZF2WPOOHGDPFXIMIV0YKEJI0N3CKE0ENHIQBYV2QKD3&client_secret=X5V42VABELHKXMVRZ05L2CVXQG53EBKQVEB1SMAZF1X2H3TE&ll=40.7900869,-73.9598295&v=20180604&query=pastry&radius=250000&limit=300'

In [69]:
results2 = requests.get(url2).json()
results2

{'meta': {'code': 200, 'requestId': '5d5f251f2b274a002c300254'},
 'response': {'venues': [{'id': '4ab79ff3f964a5203f7a20e3',
    'name': 'Corrado Bread & Pastry',
    'location': {'address': '960 Lexington Ave',
     'crossStreet': 'at E 70th St',
     'lat': 40.76909506094851,
     'lng': -73.96323265478566,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.76909506094851,
       'lng': -73.96323265478566}],
     'distance': 2354,
     'postalCode': '10021',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['960 Lexington Ave (at E 70th St)',
      'New York, NY 10021',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d16a941735',
      'name': 'Bakery',
      'pluralName': 'Bakeries',
      'shortName': 'Bakery',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
       'suffix': '.png'},
      'primary': True}],
    'delivery': {'id': '781382',
     'url':

In [70]:
venues2 = results2['response']['venues']

# tranform venues into a dataframe
dataframe2 = json_normalize(venues2)
dataframe2

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",781382,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/corrado-bread-an...,False,4ab79ff3f964a5203f7a20e3,960 Lexington Ave,...,"[960 Lexington Ave (at E 70th St), New York, N...","[{'label': 'display', 'lat': 40.76909506094851...",40.769095,-73.963233,NaN,10021,NY,Corrado Bread & Pastry,v-1566516511,NaN
1,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ee0e82161aff91575bab244,89 E 42nd St,...,[89 E 42nd St (btwn Lexington Ave & E 43rd St)...,"[{'label': 'display', 'lat': 40.75228482451162...",40.752285,-73.975849,NaN,10017,NY,Eli's Bread and Pastry,v-1566516511,NaN
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,40f32600f964a520720a1fe3,1030 Amsterdam Ave,...,[1030 Amsterdam Ave (btwn W 110th & W 111th St...,"[{'label': 'display', 'lat': 40.80356193389294...",40.803562,-73.963475,NaN,10025,NY,The Hungarian Pastry Shop,v-1566516511,NaN
3,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b0f513bf964a520756123e3,29-15 Ditmars Blvd,...,"[29-15 Ditmars Blvd (btwn 29th & 31st St), Ast...","[{'label': 'display', 'lat': 40.77597229484988...",40.775972,-73.910454,NaN,11105,NY,LaGuli Pastry Shop,v-1566516511,67536007
4,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a4bf8ccf964a520eeac1fe3,372 3rd Ave,...,"[372 3rd Ave (at 27th St.), New York, NY 10016...","[{'label': 'display', 'lat': 40.74126115220162...",40.741261,-73.981284,NaN,10016,NY,La Delice Pastry Shop,v-1566516511,NaN
5,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4c055c56f56c2d7f078a1f66,1361 Lexington Ave,...,"[1361 Lexington Ave (90th street), New York, N...","[{'label': 'display', 'lat': 40.78197554602246...",40.781976,-73.953761,NaN,10128,NY,Corrado Bread & Pastry,v-1566516511,NaN
6,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a75f1d7f964a520c1e11fe3,298 Court St,...,"[298 Court St (at Degraw St), Brooklyn, NY 112...","[{'label': 'display', 'lat': 40.684067, 'lng':...",40.684067,-73.995377,NaN,11231,NY,Court Pastry Shop,v-1566516511,NaN
7,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4d865f51e83fa14303c28ea1,24 S Broad St,...,"[24 S Broad St, Ridgewood, NJ 07450, United St...","[{'label': 'display', 'lat': 40.97911421644311...",40.979114,-74.120686,NaN,07450,NJ,Sook Pastry Shop,v-1566516511,41416272
8,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ab52b10f964a520f37220e3,670 E 187th St,...,"[670 E 187th St, Bronx, NY 10458, United States]","[{'label': 'display', 'lat': 40.85432460487509...",40.854325,-73.885112,NaN,10458,NY,Artuso Pastry Shop,v-1566516511,NaN
9,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bbcf74f8a4fb713ff693c9d,256 Ridge Road,...,"[256 Ridge Road, Lyndhurst, NJ 07071, United S...","[{'label': 'display', 'lat': 40.8099089316324,...",40.809909,-74.117547,NaN,07071,NJ,Lyndhurst Pastry Shop,v-1566516511,511495036


In [71]:
filtered_columns2 = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered2 = dataframe2.loc[:, filtered_columns2]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered2['categories'] = dataframe_filtered2.apply(get_category_type, axis=1)

dataframe_filtered2.columns = [column.split('.')[-1] for column in dataframe_filtered2.columns]

#results from only "NEW YORK" will show up. In this case, New York refers to Manhattan
indexNames = dataframe_filtered2[dataframe_filtered2['city'] !="New York" ].index
dataframe_filtered2.drop(indexNames, inplace=True)
dataframe_filtered2=dataframe_filtered2[['name', 'categories', 'address', 'city', 'country', 'distance', 'lat', 'lng']]


dataframe_filtered2

,name,categories,address,city,country,distance,lat,lng
0,Corrado Bread & Pastry,Bakery,960 Lexington Ave,New York,United States,2354,40.769095,-73.963233
1,Eli's Bread and Pastry,Bakery,89 E 42nd St,New York,United States,4419,40.752285,-73.975849
2,The Hungarian Pastry Shop,Coffee Shop,1030 Amsterdam Ave,New York,United States,1531,40.803562,-73.963475
4,La Delice Pastry Shop,Bakery,372 3rd Ave,New York,United States,5728,40.741261,-73.981284
5,Corrado Bread & Pastry,Café,1361 Lexington Ave,New York,United States,1037,40.781976,-73.953761
19,Royale Cafe & Pastry,Café,461 Avenue of the Americas,New York,United States,6901,40.735341,-73.998239
21,Santi's Pastry Shop,Residential Building (Apartment / Condo),223 W 115th St,New York,United States,1557,40.803403,-73.954173
23,Bread & Pastry Cafe,Café,330 Bleecker St,New York,United States,7333,40.733530,-74.004423
25,B.M. Place Best Coffee & Pastry,Coffee Shop,115-153 E 75th St,New York,United States,1965,40.772452,-73.960921
34,Sweet Life Pastry,Bakery,3887 Broadway,New York,United States,5473,40.837456,-73.942427


In [75]:
address3 = 'Queens'
geolocator3 = Nominatim(user_agent="foursquare_agent")
location3 = geolocator3.geocode(address3)
latitude3 = location3.latitude
longitude3 = location3.longitude

CLIENT_ID = '2AK23ZF2WPOOHGDPFXIMIV0YKEJI0N3CKE0ENHIQBYV2QKD3' # your Foursquare ID
CLIENT_SECRET = 'X5V42VABELHKXMVRZ05L2CVXQG53EBKQVEB1SMAZF1X2H3TE' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 300
search_query3 = 'pastry'
radius3 = 2500000 #length of Manhattan. I corrected for any outliers in case some results were outside Manhattan.

In [76]:
url3='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude3, longitude3, VERSION, search_query3, radius3, LIMIT)
url3

'https://api.foursquare.com/v2/venues/search?client_id=2AK23ZF2WPOOHGDPFXIMIV0YKEJI0N3CKE0ENHIQBYV2QKD3&client_secret=X5V42VABELHKXMVRZ05L2CVXQG53EBKQVEB1SMAZF1X2H3TE&ll=40.6524927,-73.7914214158161&v=20180604&query=pastry&radius=2500000&limit=300'

In [77]:
results3 = requests.get(url3).json()
results3

{'meta': {'code': 200, 'requestId': '5d5f253c00bad70025ddac03'},
 'response': {'venues': [{'id': '4a75f1d7f964a520c1e11fe3',
    'name': 'Court Pastry Shop',
    'location': {'address': '298 Court St',
     'crossStreet': 'at Degraw St',
     'lat': 40.684067,
     'lng': -73.995377,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.684067,
       'lng': -73.995377}],
     'distance': 17575,
     'postalCode': '11231',
     'cc': 'US',
     'city': 'Brooklyn',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['298 Court St (at Degraw St)',
      'Brooklyn, NY 11231',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d16a941735',
      'name': 'Bakery',
      'pluralName': 'Bakeries',
      'shortName': 'Bakery',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1566516540',
    'hasPerk': False},
   {'id': '4b0f513bf964a520756123

In [78]:
venues3 = results3['response']['venues']

# tranform venues into a dataframe
dataframe3 = json_normalize(venues3)


In [79]:
filtered_columns3 = ['name', 'categories'] + [col for col in dataframe3.columns if col.startswith('location.')] + ['id']
dataframe_filtered3 = dataframe.loc[:, filtered_columns3]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered3['categories'] = dataframe_filtered3.apply(get_category_type, axis=1)

dataframe_filtered3.columns = [column.split('.')[-1] for column in dataframe_filtered3.columns]
#checks for only results from Queens

indexNames3 = dataframe_filtered3[dataframe_filtered3['city'] !="Queens" ].index
dataframe_filtered3.drop(indexNames3, inplace=True)
#getting rid of extraneous categories
dataframe_filtered3=dataframe_filtered3[['name', 'categories', 'address', 'city', 'country', 'distance', 'lat', 'lng']]

dataframe_filtered3

,name,categories,address,city,country,distance,lat,lng
10,Ciro's Pastry,Bakery,Beach 129th st,Queens,United States,11643,40.578328,-73.849355


In [97]:
address4 = 'The Bronx'
geolocator4 = Nominatim(user_agent="foursquare_agent")
location4 = geolocator4.geocode(address4)
latitude4 = location4.latitude
longitude4 = location4.longitude

CLIENT_ID = '2AK23ZF2WPOOHGDPFXIMIV0YKEJI0N3CKE0ENHIQBYV2QKD3' #
CLIENT_SECRET = 'X5V42VABELHKXMVRZ05L2CVXQG53EBKQVEB1SMAZF1X2H3TE'
VERSION = '20180604'
LIMIT = 300
search_query4 = 'pastry'
radius4 = 250000 #length of Manhattan. I corrected for any outliers in case some results were outside Manhattan.

In [98]:
url4='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude4, longitude4, VERSION, search_query4, radius4, LIMIT)
url4

'https://api.foursquare.com/v2/venues/search?client_id=2AK23ZF2WPOOHGDPFXIMIV0YKEJI0N3CKE0ENHIQBYV2QKD3&client_secret=X5V42VABELHKXMVRZ05L2CVXQG53EBKQVEB1SMAZF1X2H3TE&ll=40.85048545,-73.8404035580209&v=20180604&query=pastry&radius=250000&limit=300'

In [99]:
results4 = requests.get(url4).json()
results4
venues4 = results4['response']['venues']

# tranform venues into a dataframe
dataframe4 = json_normalize(venues4)

In [102]:
filtered_columns4 = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered4 = dataframe.loc[:, filtered_columns4]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered4['categories'] = dataframe_filtered4.apply(get_category_type, axis=1)

# filtered results so only Bronx appears
dataframe_filtered4.columns = [column.split('.')[-1] for column in dataframe_filtered4.columns]
indexNames4 = dataframe_filtered4[dataframe_filtered4['city'] !="Bronx" ].index
dataframe_filtered4.drop(indexNames4, inplace=True)
dataframe_filtered4=dataframe_filtered4[['name', 'categories', 'address', 'city', 'country', 'distance', 'lat', 'lng']]

dataframe_filtered4

,name,categories,address,city,country,distance,lat,lng
16,Artuso Pastry Shop,Bakery,670 E 187th St,Bronx,United States,23374,40.854325,-73.885112
20,DeLillo Pastry Shop,Dessert Shop,606 E 187th St,Bronx,United States,23447,40.855364,-73.887198
27,Morrone Pastry Shop & Cafe,Dessert Shop,2349 Arthur Ave,Bronx,United States,23345,40.854636,-73.888400


In [92]:
address5 = 'Staten Island'
geolocator5 = Nominatim(user_agent="foursquare_agent")
location5 = geolocator5.geocode(address5)
latitude5 = location5.latitude
longitude5 = location5.longitude

CLIENT_ID = '2AK23ZF2WPOOHGDPFXIMIV0YKEJI0N3CKE0ENHIQBYV2QKD3' #  Foursquare ID
CLIENT_SECRET = 'X5V42VABELHKXMVRZ05L2CVXQG53EBKQVEB1SMAZF1X2H3TE' 
VERSION = '20180604'
LIMIT = 300
search_query5 = 'pastry' #search querty
radius5 = 2500000

In [93]:
url5='https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude5, longitude5, VERSION, search_query5, radius5, LIMIT)
url5

'https://api.foursquare.com/v2/venues/search?client_id=2AK23ZF2WPOOHGDPFXIMIV0YKEJI0N3CKE0ENHIQBYV2QKD3&client_secret=X5V42VABELHKXMVRZ05L2CVXQG53EBKQVEB1SMAZF1X2H3TE&ll=40.5834557,-74.1496048&v=20180604&query=pastry&radius=2500000&limit=300'

In [94]:
results5 = requests.get(url5).json()
venues5 = results5['response']['venues']
venues5
dataframe5 = json_normalize(venues5)
dataframe5

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a75f1d7f964a520c1e11fe3,298 Court St,...,"[298 Court St (at Degraw St), Brooklyn, NY 112...","[{'label': 'display', 'lat': 40.684067, 'lng':...",40.684067,-73.995377,NaN,11231,NY,Court Pastry Shop,v-1566516612,NaN
1,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ccae33cc4d06dcbebeb6503,1646 Forest Ave,...,"[1646 Forest Ave, Staten Island, NY 10302, Uni...","[{'label': 'display', 'lat': 40.62481841551805...",40.624818,-74.143288,NaN,10302,NY,Renato's Imperial Pastry Shoppe,v-1566516612,NaN
2,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b898727f964a5200b3f32e3,101 East Broad Street,...,"[101 East Broad Street, Westfield, NJ 07090, U...","[{'label': 'display', 'lat': 40.65070726511008...",40.650707,-74.349392,NaN,07090,NJ,Bovella’s Pastry Shoppe,v-1566516612,151955903
3,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b9c11bef964a520f04636e3,8218 13th Ave,...,"[8218 13th Ave (83rd St), Brooklyn, NY 11228, ...","[{'label': 'display', 'lat': 40.61605170077949...",40.616052,-74.012058,NaN,11228,NY,Tasty Pastry,v-1566516612,NaN
4,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ee0e82161aff91575bab244,89 E 42nd St,...,[89 E 42nd St (btwn Lexington Ave & E 43rd St)...,"[{'label': 'display', 'lat': 40.75228482451162...",40.752285,-73.975849,NaN,10017,NY,Eli's Bread and Pastry,v-1566516612,NaN
5,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4a4bf8ccf964a520eeac1fe3,372 3rd Ave,...,"[372 3rd Ave (at 27th St.), New York, NY 10016...","[{'label': 'display', 'lat': 40.74126115220162...",40.741261,-73.981284,NaN,10016,NY,La Delice Pastry Shop,v-1566516612,NaN
6,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",23728,/delivery_provider_seamless_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",seamless,https://www.seamless.com/menu/jean-danet-pastr...,False,4a3fbddcf964a520e6a31fe3,7526 5th Ave,...,"[7526 5th Ave (at 76th St), Brooklyn, NY 11209...","[{'label': 'display', 'lat': 40.62883876030009...",40.628839,-74.023006,NaN,11209,NY,"Jean Danet Pastry, Cafe and Brick Oven Pizza",v-1566516612,74057958
7,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4dfb7c9d52b145696e36ea1a,4366 Amboy Rd,...,"[4366 Amboy Rd (4366 Amboy Rd), Staten Island,...","[{'label': 'display', 'lat': 40.54503688560972...",40.545037,-74.160852,NaN,10312,NY,Alfonso's Pastry Shop,v-1566516612,NaN
8,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4b0f513bf964a520756123e3,29-15 Ditmars Blvd,...,"[29-15 Ditmars Blvd (btwn 29th & 31st St), Ast...","[{'label': 'display', 'lat': 40.77597229484988...",40.775972,-73.910454,NaN,11105,NY,LaGuli Pastry Shop,v-1566516612,67536007
9,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bbcf74f8a4fb713ff693c9d,256 Ridge Road,...,"[256 Ridge Road, Lyndhurst, NJ 07071, United S...","[{'label': 'display', 'lat': 40.8099089316324,...",40.809909,-74.117547,NaN,07071,NJ,Lyndhurst Pastry Shop,v-1566516612,511495036


In [95]:
filtered_columns5 = ['name', 'categories'] + [col for col in dataframe5.columns if col.startswith('location.')] + ['id']
dataframe_filtered5 = dataframe5.loc[:, filtered_columns5]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe_filtered5['categories'] = dataframe_filtered5.apply(get_category_type, axis=1)

# clean column names 
dataframe_filtered5.columns = [column.split('.')[-1] for column in dataframe_filtered5.columns]

indexNames5 = dataframe_filtered5[dataframe_filtered5['city'] !="Staten Island" ].index
#only get results that say shop is in Staten Island

dataframe_filtered5.drop(indexNames5, inplace=True)
dataframe_filtered5=dataframe_filtered5[['name', 'categories', 'address', 'city', 'country', 'distance', 'lat', 'lng']]


dataframe_filtered5

,name,categories,address,city,country,distance,lat,lng
1,Renato's Imperial Pastry Shoppe,Bakery,1646 Forest Ave,Staten Island,United States,4635,40.624818,-74.143288
7,Alfonso's Pastry Shop,Bakery,4366 Amboy Rd,Staten Island,United States,4381,40.545037,-74.160852
11,La Dolce Bakery & Pastry Shoppe,Bakery,6321 Amboy Rd,Staten Island,United States,8745,40.523312,-74.216123
13,La Dolce Bakery & Pastry Shoppe,Bakery,645 Rossville Ave,Staten Island,United States,6620,40.542735,-74.206663
20,Alfonso's Pastry Shoppe,Dessert Shop,1899 Victory Blvd,Staten Island,United States,3773,40.612822,-74.127298
21,Amici's Bakery & Pastry Shoppe,Bakery,Arthur Kill Road,Staten Island,United States,2898,40.560513,-74.165819
29,Rispoli Pastry Shop,Bakery,1184 Bay St,Staten Island,United States,7825,40.614347,-74.066437
32,Golden Pastry Shop,Bakery,1 Corson Ave,Staten Island,United States,8549,40.638281,-74.078757


In [103]:
totaldataframe=merged_df = pd.concat([dataframe_filtered, dataframe_filtered2, dataframe_filtered3, dataframe_filtered4, dataframe_filtered5])
totaldataframe

,name,categories,address,city,country,distance,lat,lng
0,Court Pastry Shop,Bakery,298 Court St,Brooklyn,United States,5407,40.684067,-73.995377
1,Aliotta Pastry Shop,Bakery,4522 Avenue N,Brooklyn,United States,3836,40.618712,-73.930852
4,Tasty Pastry,Bakery,8218 13th Ave,Brooklyn,United States,6498,40.616052,-74.012058
6,Circo's Pastry Shop,Bakery,312 Knickerbocker Ave,Brooklyn,United States,6141,40.701525,-73.923215
8,"Jean Danet Pastry, Cafe and Brick Oven Pizza",Bakery,7526 5th Ave,Brooklyn,United States,6638,40.628839,-74.023006
9,Peter Pan Donut & Pastry Shop,Donut Shop,727 Manhattan Ave,Brooklyn,United States,8463,40.726102,-73.952252
11,I.M. Pastry Studio,Bakery,1131 Nostrand Ave,Brooklyn,United States,1098,40.659950,-73.950523
13,Circo's Pastry Shop,None,312 Knickerbocker Ave,Brooklyn,United States,1385,40.640000,-73.940000
17,Daynes Pastry,Bakery,101 Lincoln Rd,Brooklyn,United States,1464,40.660999,-73.959297
18,Damascus Bread & Pastry Shop,Bakery,195 Atlantic Ave,Brooklyn,United States,5765,40.690047,-73.993054


This is the dataframe of all the pastry shops all 5 boroughs of NYC. On the map, I am going to plot the centers of each borough and the surrounding pastry shops to give a general idea of which borough would be ideal for placing a pastry shop.

In [ ]:
venues_map = folium.Map(location=[40.730610, -73.935242],  zoom_start=11) # generate map centred around Ecco


#Manhattan Center

folium.features.CircleMarker(
        [40.7831, -73.9712],
        radius=10,
        popup='Manhattan',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
).add_to(venues_map)
    
    
  #Brooklyn Center

folium.features.CircleMarker(
        [40.6782, -73.9442],
        radius=10,
        popup='Brooklyn',
        fill=True,
        color='yellow',
        fill_color='yellow',
        fill_opacity=0.6
).add_to(venues_map)
    
    #The Bronx Center
    
folium.features.CircleMarker(
        [40.8448, -73.8648],
        radius=10,
        popup='Bronx',
        fill=True,
        color='green',
        fill_color='green',
        fill_opacity=0.6
).add_to(venues_map)
   
   #Queens Center
folium.features.CircleMarker(
        [40.7282, -73.7949],
        radius=10,
        popup='Bronx',
        fill=True,
        color='orange',
        fill_color='orange',
        fill_opacity=0.6
).add_to(venues_map)
        
        
       #Staten Island Center
folium.features.CircleMarker(
        [40.5795, -74.1502],
        radius=10,
        popup='Staten Island',
        fill=True,
        color='pink',
        fill_color='pink',
        fill_opacity=0.6
).add_to(venues_map)

# added pastry shops to map



for lat, lng, label in zip(totaldataframe.lat, totaldataframe.lng, totaldataframe.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

The map shows all of the locations of the pastry shops throughout NYC. From the datatable and also the map,  we see Queens and the Bronx has the lowest amount of pastry shops. Lets see what would be some great choices in the neighborhoods of Queens and Bronx to place pastry shop.

In [133]:
df_data_2=df_data_1[df_data_1.BOROUGH=="QUEENS"]
df_data_3=df_data_1[df_data_1.BOROUGH=='THE BRONX']
queenbronx=df_data_2.append(df_data_3)
queenbronx.sort_values(by='$ PER SQ FT', ascending=True, inplace=True)

queenbronx

,BOROUGH,NEIGHBORHOOD,$ PER SQ FT
15,QUEENS,Auburndale,1.03
10,THE BRONX,Morris Heights,1.12
16,QUEENS,Bay Terrace,1.44
17,QUEENS,Ozone Park,1.67
11,THE BRONX,Fordham Manor & Williamsbridge,1.80
12,THE BRONX,Morris Park,1.88
13,THE BRONX,Pelham Gardens,1.89
18,QUEENS,Rockaway Beach,1.91
14,THE BRONX,Pelham Bay,1.93
19,QUEENS,Maspeth,2.04


The first four data entries in queenbronx would be great locations for the new pastry shop as they are lower in price. Ideally, Queens, perhaps Auburndale, would be a great place to build the shop because Queens had the lowest number of pastry shops. I sorted the boroughs by price per square feet in USD to give the potential pastry bakery owner a better idea.